# PENGENALAN

--------------------------
Nama : Handwitanto Abraham
--------------------------
Batch : RMT_036
--------------------------

## Membuat koneksi dengan BigQuery

In [50]:
#syntax ini digunakan untuk melakukan import auth dan bigquery, serta melakukan autentikasi pada bigQuery
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = "hacktiv8-435414" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id_akun)

Authenticated


## Loading data awal

In [77]:
#syntax ini digunakan untuk memanggil data pada dataset
query = """SELECT*FROM`ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`

"""

df = client.query(query).to_dataframe()
df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027)
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008)
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.00,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014)
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
...,...,...,...,...,...,...,...,...,...,...,...
9281,FI-2165,I-407,2021-04-06 16:00:00+00:00,2021-04-06 16:00:00+00:00,1.287472,5.25,1.31,Credit Card,Federway Int.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259)
9282,FI-3890,I-580,2022-11-13 15:30:00+00:00,2022-11-13 15:45:00+00:00,2.574944,7.75,1.31,Credit Card,Federway Int.,POINT (-87.6173580061 41.859349715),POINT (-87.6209929134 41.8849871918)
9283,AI-11438,AI-527,2021-05-14 15:30:00+00:00,2021-05-14 15:30:00+00:00,1.448406,5.25,1.31,Credit Card,Atomic Inc.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259)
9284,AI-11220,A-842,2021-04-30 18:00:00+00:00,2021-04-30 18:00:00+00:00,1.770274,6.25,1.56,Credit Card,Atomic Inc.,POINT (-87.6327464887 41.8809944707),POINT (-87.657005027 41.8790669938)


#### Insight :
- Terdapat 11 kolom data, dan 9287 baris didalamnya yang berisi informasi terkait perjalanan taksi, biaya, dan perusaahaan terkait.
- Data sudah tersimpan dengan cukup rapih, beberapa pengulangan terjadi pada kolom Company, Payment_type sehingga dapat dilakukan partisi untuk mengoptimasi database

## Problem 1 - SQL


### 1. Apa saja nama perusahaan yang terdapat dalam data?

In [78]:
#syntax ini akan memanggil unique value pada kolom 'company' sehingga dapat terlihat nama-nama perusahaan yang ada pada data
query = """SELECT DISTINCT (company) FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`

"""
df = client.query(query).to_dataframe()
df

,company
0,Deluc Porter
1,Synctaxi Inc.
2,Atomic Inc.
3,Federway Int.
4,Rocket Cab Auto


#### Insight :
- dari 9000+ data yang ada, ternyata hanya ada 5 perusahaan yang tercakup di dalamnya, sehingga diperlukan normalization data

### 2. Berapa nilai minimum dan maksimum dari waktu pengambilan data?

In [81]:
query = """
SELECT *,
  EXTRACT(HOUR FROM trip_start_timestamp) AS jam_a,
  EXTRACT(MINUTE FROM trip_start_timestamp) AS jam_b,
  EXTRACT(HOUR FROM trip_end_timestamp) AS jam_c,
  EXTRACT(MINUTE FROM trip_end_timestamp) AS jam_d

FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`

"""

df = client.query(query).to_dataframe()
df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location,jam_a,jam_b,jam_c,jam_d
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),19,30,19,45
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027),15,15,15,45
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008),19,30,19,45
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.00,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014),13,45,14,15
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),15,15,15,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,FI-2165,I-407,2021-04-06 16:00:00+00:00,2021-04-06 16:00:00+00:00,1.287472,5.25,1.31,Credit Card,Federway Int.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259),16,0,16,0
9282,FI-3890,I-580,2022-11-13 15:30:00+00:00,2022-11-13 15:45:00+00:00,2.574944,7.75,1.31,Credit Card,Federway Int.,POINT (-87.6173580061 41.859349715),POINT (-87.6209929134 41.8849871918),15,30,15,45
9283,AI-11438,AI-527,2021-05-14 15:30:00+00:00,2021-05-14 15:30:00+00:00,1.448406,5.25,1.31,Credit Card,Atomic Inc.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259),15,30,15,30
9284,AI-11220,A-842,2021-04-30 18:00:00+00:00,2021-04-30 18:00:00+00:00,1.770274,6.25,1.56,Credit Card,Atomic Inc.,POINT (-87.6327464887 41.8809944707),POINT (-87.657005027 41.8790669938),18,0,18,0


### 3. Memberikan gambaran jumlah perjalanan setiap perusahaan pada tahun 2022
Dapat digunakan untuk menganalisa perkembangan pasar dan perusahaan yang diminati



In [7]:
#syntax ini digunakan untuk memanggil value dari company, menghitung jumlah value yang sama pada kolom company, dan menampilkan hasilnya hanya untuk tahun 2022 dengan penggunaan WHERE, dimana tahun didapat dengan meng-extract data pada kolom start_trip_timestamp
#kemudian di kelompokkan kembali berdasarkan 'company'

query = """SELECT company,COUNT (company) AS jumlah_perjalanan FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`
  WHERE EXTRACT(YEAR from trip_start_timestamp) = 2022
  GROUP BY company

"""

df = client.query(query).to_dataframe()
df

,company,jumlah_perjalanan
0,Deluc Porter,1050
1,Synctaxi Inc.,1341
2,Federway Int.,1059
3,Rocket Cab Auto,77


#### Insight :
- Pada tahun 2022, hanya ada 4 perusahaan taxi yang masih beroperasi
- Synctaxi Inc. adalah perusahaan taxi yang melakukan paling banyak perjalanan pada tahun 2022

### 4. Menentukan perusahaan yang memiliki tarif rata-rata paling rendah
Dapat digunakan untuk menganalisa korelasi antara tarif dengan jumlah perjalanan yang dilakukan

In [16]:
#Syntax ini digunakan untuk mengetahui tarif terendah dengan perhitungan trips_km / fare, kemudian dikelompokkan berdasarkan company dan diambil nilai rata-ratanya
#Setelah mendapat angka rata-rata dari seluruh perusahaan, di urutkan berdasarkan angka yang terendah, dan hanya menampilkan urutan yang pertama (terendah)
query = """
SELECT company, AVG (biaya_per_km) AS tarif_rata_rata FROM
(SELECT company, SAFE_DIVIDE (fare, ROUND(trip_km)) AS biaya_per_km FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`)
GROUP BY company
ORDER BY tarif_rata_rata ASC
LIMIT 1

"""

df = client.query(query).to_dataframe()
df

,company,tarif_rata_rata
0,Rocket Cab Auto,2.260277


#### Insight :
- Peruhsaan taxi dengan tarif terendah adalah Rocket Cab Auto

## Problem 2 - Pandas

In [51]:
#syntax ini digunakan untuk memanggil data

import pandas as pd

syn = """SELECT*FROM`ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`

"""

df = client.query(syn).to_dataframe()
df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027)
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008)
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.00,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014)
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
...,...,...,...,...,...,...,...,...,...,...,...
9281,FI-2165,I-407,2021-04-06 16:00:00+00:00,2021-04-06 16:00:00+00:00,1.287472,5.25,1.31,Credit Card,Federway Int.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259)
9282,FI-3890,I-580,2022-11-13 15:30:00+00:00,2022-11-13 15:45:00+00:00,2.574944,7.75,1.31,Credit Card,Federway Int.,POINT (-87.6173580061 41.859349715),POINT (-87.6209929134 41.8849871918)
9283,AI-11438,AI-527,2021-05-14 15:30:00+00:00,2021-05-14 15:30:00+00:00,1.448406,5.25,1.31,Credit Card,Atomic Inc.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259)
9284,AI-11220,A-842,2021-04-30 18:00:00+00:00,2021-04-30 18:00:00+00:00,1.770274,6.25,1.56,Credit Card,Atomic Inc.,POINT (-87.6327464887 41.8809944707),POINT (-87.657005027 41.8790669938)


### 1. Melakukan klasifikasi jam perjalanan berdasarkan beberapa periode
Dapat digunakan untuk menentukan periode terpadat pengguna taxi

In [58]:
#syntax ini digunakan untuk meng-extract data Jam pada trip_start_timestamp, serta membuat kolom 'jam' dan 'keterangan_waktu'
df['jam'] = df['trip_start_timestamp'].dt.hour
df['keterangan_waktu'] = ''

#Looping ini digunakan untuk menentukan kategori jam sesuai dengan persyaratan
for idx, row in df.iterrows():
  jam = row['jam']
  if jam < 4:
      df.at[idx, 'keterangan_waktu'] = 'Dini Hari'
  elif jam < 10:
      df.at[idx, 'keterangan_waktu'] = 'Pagi'
  elif jam < 16:
      df.at[idx, 'keterangan_waktu'] = 'Siang'
  elif jam < 19:
      df.at[idx, 'keterangan_waktu'] = 'Sore'
  elif jam < 24:
      df.at[idx, 'keterangan_waktu'] = 'Malam'

df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location,jam,keterangan_waktu,year
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),19,Malam,2021
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027),15,Siang,2021
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008),19,Malam,2021
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.00,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014),13,Siang,2021
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),15,Siang,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,FI-2165,I-407,2021-04-06 16:00:00+00:00,2021-04-06 16:00:00+00:00,1.287472,5.25,1.31,Credit Card,Federway Int.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259),16,Sore,2021
9282,FI-3890,I-580,2022-11-13 15:30:00+00:00,2022-11-13 15:45:00+00:00,2.574944,7.75,1.31,Credit Card,Federway Int.,POINT (-87.6173580061 41.859349715),POINT (-87.6209929134 41.8849871918),15,Siang,2022
9283,AI-11438,AI-527,2021-05-14 15:30:00+00:00,2021-05-14 15:30:00+00:00,1.448406,5.25,1.31,Credit Card,Atomic Inc.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259),15,Siang,2021
9284,AI-11220,A-842,2021-04-30 18:00:00+00:00,2021-04-30 18:00:00+00:00,1.770274,6.25,1.56,Credit Card,Atomic Inc.,POINT (-87.6327464887 41.8809944707),POINT (-87.657005027 41.8790669938),18,Sore,2021


### 2. Menentukan total jarak perjalanan oleh Federway Int. sepanjang tahun 2022



In [57]:
#syntax ini digunakan untuk mengambil data pada dataframe dengan ketentuan :
# - nama perusahaan harus "Federway Int."
# - data khusus untuk tahun 2022 (didapat dengan melakukan extract data pada kolom 'start_date_timestamp'
# - melakukan aggregation dengan menghitung jumlah kolom 'trip_km' pada data yang dipanggil

df['year'] = df['trip_start_timestamp'].dt.year
df.query('company == "Federway Int." and year == 2022').agg({'trip_km':'sum'})


,0
trip_km,4359.24135


## KESIMPULAN
Data Frame ini menampilkan data gabungan beberapa perusahaan taksi, dan memberikan informasi seperti :
- ID Taxi
- Durasi Perjalanan
- Jarak Perjalanan
- Tarif Perjalan

Dari data ini, dapat dibuat analisa lebih detil terkait banyak hal, diantaranya : menentukan jam sibuk, periode dengan pendapatan tertinggi, jumlah rata-rata pengguna taxi, dan sebagainya